In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, timedelta

### Google
Source: https://www.google.com/covid19/mobility/

In [2]:

#Google
pathGoogle = "../data/mobility/raw_data/MobilityTrends/Google/"
df_daily = pd.read_csv(pathGoogle + 'Global_Mobility_Report.csv', encoding='latin-1', delimiter=',')
df_daily = df_daily[df_daily['country_region_code'] == 'DE'] #reduce to Germany
df_daily = df_daily[df_daily['sub_region_1'].isnull()] #reduce to only Germany and not Bundesländer

#drop some unused columns
df_daily = df_daily.drop(['sub_region_1', 'sub_region_2', 'country_region_code', 'country_region', 'iso_3166_2_code', 'census_fips_code'], axis = 1)

#shorten column name
for column in df_daily:
    if '_percent_change_from_baseline' in column:
        df_daily.rename(columns={column:column.replace('_percent_change_from_baseline','')},inplace=True)

#rename columns to our standard
df_daily.rename(columns={'retail_and_recreation': 'M_GoogleMR_RetailRecreation',
                        'grocery_and_pharmacy': 'M_GoogleMR_GroceryPharmacy',
                        'parks': 'M_GoogleMR_Parks',
                        'transit_stations': 'M_GoogleMR_TransitStations',
                        'workplaces': 'M_GoogleMR_Workplaces',
                        'residential': 'M_GoogleMR_Residential'
                        }, inplace=True)

#process monthly means    
df_monthly = df_daily
df_daily = df_daily.set_index('date')
df_monthly.date = pd.to_datetime(df_monthly.date).dt.to_period('m')
df_monthly = df_monthly.groupby('date').mean()

#create ratio values instead of prozentual
df_daily = df_daily.div(100)
df_monthly = df_monthly.div(100)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#save daily dataframe
df_daily.to_csv('../data/mobility/raw_data/MobilityTrends/Google/GoogleMobilityReport_daily2020.csv')
#save monthly dataframe
df_monthly.to_csv('../data/mobility/GoogleMobilityReport_monthly.csv')

### Apple
Source: https://www.apple.com/covid19/mobility

In [4]:
#Apple
#read Apple data and reduce it to Germany and driving
pathApple = "../data/mobility/raw_data/MobilityTrends/Apple/"
df_daily = pd.read_csv(pathApple + 'applemobilitytrends-2020-06-23.csv', encoding='latin-1', delimiter=',')
df_daily = df_daily[df_daily['region'] == 'Germany']
df_daily = df_daily[df_daily['transportation_type'] == 'driving'] 

#drop some unused columns
df_daily = df_daily.drop(['geo_type', 'region', 'transportation_type', 'alternative_name', 'sub-region', 'country'], axis=1)

#transpose dataframe
df_daily = pd.melt(df_daily, var_name="date", value_name="M_AppleMobRep_DiffRatioDriving")

#reference baseline is in Apple data 100% but we are comparing at a reference line of 0%
df_daily.M_AppleMobRep_DiffRatioDriving = df_daily.M_AppleMobRep_DiffRatioDriving - 100

#process to monthly means
df_monthly = df_daily
df_daily = df_daily.set_index('date')

df_monthly.date = pd.to_datetime(df_monthly.date).dt.to_period('m')
df_monthly = df_monthly.groupby('date').mean()

#create ratio values instead of prozentual
df_daily = df_daily.div(100)
df_monthly = df_monthly.div(100)

In [5]:
#save daily dataframe
df_daily.to_csv('../data/mobility/raw_data/MobilityTrends/Apple/AppleMobilityReport_daily2020.csv')
#save monthly dataframe
df_monthly.to_csv('../data/mobility/AppleMobilityReport_monthly.csv')

### BAST
Source: https://www.bast.de/BASt_2017/DE/Statistik/Verkehrsdaten/Verkehrsbarometer.html

In [2]:
#alues from downloaded pdf taken manually and stored in raw_data in a excel file. Uncommenting and changig this code makes it able to update the data

df_dates = pd.read_excel('../data/mobility/raw_data/MobilityTrends/BAST/MobilityTrends_BAST_rawdata.xls')

#KFZ
val_BAST_KFZ = np.array(df_dates.val_BAST_KFZ.values)
val_BAST_SV = np.array(df_dates.val_BAST_SV.values)
val_BAST_LV = np.array(df_dates.val_BAST_LV.values)

#dataframe for interpolation
df_dates.date_start = pd.to_datetime(df_dates.date_start)
df_dates.date_ende = pd.to_datetime(df_dates.date_ende)

#interpolate weeks, so every day of the week has same value
dates = []
val_KFZ = []
val_SV = []
val_LV = []
row, col = df_dates.shape
for r in np.arange(0,row):
    delta = df_dates.date_ende[r] - df_dates.date_start[r]
    for i in range(delta.days):
        dates.append(df_dates.date_start[r] + timedelta(days=i))
        val_KFZ.append(df_dates.val_BAST_KFZ[r])
        val_SV.append(df_dates.val_BAST_SV[r])
        val_LV.append(df_dates.val_BAST_LV[r])

#generate daily dataframe
df_daily = pd.DataFrame({"date":dates, 
                         "M_BASTMobRep_DiffRatio_KFZ":val_KFZ,
                         "M_BASTMobRep_DiffRatio_SV":val_SV,
                         "M_BASTMobRep_DiffRatio_LV":val_LV,
                        }, index=dates) #, index=date


#process to monthly means
df_monthly = df_daily
df_monthly.date = pd.to_datetime(df_monthly.date).dt.to_period('m')
df_monthly = df_monthly.groupby('date').mean()

df_daily = df_daily.set_index('date')

#create ratio values instead of prozentual
df_daily = df_daily.div(100)
df_monthly = df_monthly.div(100)

In [3]:
df_monthly.head(10)

,M_BASTMobRep_DiffRatio_KFZ,M_BASTMobRep_DiffRatio_SV,M_BASTMobRep_DiffRatio_LV
date,,,
2020-03,-0.435000,-0.065000,-0.500000
2020-04,-0.455294,-0.252941,-0.491765
2020-05,-0.218148,-0.136296,-0.231852
2020-06,-0.074000,-0.045600,-0.076000


In [4]:
#save daily dataframe
df_daily.to_csv('../data/mobility/raw_data/MobilityTrends/BAST/BASTMobilityReport_daily.csv')
#save monthly dataframe
df_monthly.to_csv('../data/mobility/BASTMobilityReport_monthly.csv')